In [1]:
%reload_ext autoreload
%autoreload 2

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [12]:
from types import SimpleNamespace
from typing import Any, Dict, List, Optional

from recon.dataset import Dataset
from pydantic import root_validator
from recon.types import Example, Span, Token
import numpy as np
from recon.augmentation import augment_example
from recon.operations import operation, registry

In [13]:
@operation("recon.v1.augment.kb_expansion", factory=True)
def kb_expansion(
    example: Example,
    preprocessed_outputs: Dict[str, Any] = {},
    n_augs: int = 1,
    sub_prob: float = 0.5,
) -> List[Example]:

    spans_to_aliases_map = preprocessed_outputs["recon.v1.span_aliases"]
    print(preprocessed_outputs)
    print(example.data)

    def augmentation(span: Span, spans_to_aliases_map: Dict[int, List[str]]) -> Optional[str]:
        if hash(span) in spans_to_aliases_map:
            aliases = spans_to_aliases_map[hash(span)]

            if len(aliases) > 0:
                rand_alias = np.random.choice(aliases)
                index = aliases.index(rand_alias)
                del spans_to_aliases_map[hash(span)][index]
                return rand_alias

    return augment_example(
        example,
        augmentation,
        n_augs=n_augs,
        sub_prob=sub_prob,
        spans_to_aliases_map=spans_to_aliases_map,
    )

In [14]:
factory = registry.operation_factories.get("recon.v1.augment.kb_expansion")

In [15]:
factory

<function recon.operations.operation.__call__.<locals>.factory(pre:List[recon.preprocess.PreProcessor]) -> recon.operations.Operation>

In [16]:
from recon.preprocess import SpanAliasesPreProcessor
from recon.types import Entity


entity = Entity(name="text", aliases=["new text"])
pre = [SpanAliasesPreProcessor(entities=[entity])]

In [17]:
op = factory(pre)

In [18]:
op.name, op.pre

('recon.v1.augment.kb_expansion',
 [<recon.preprocess.SpanAliasesPreProcessor at 0x7f6d9b0da7f0>])

In [29]:
e = Example(text='This is text', spans=[Span(text="text", start=8, end=12, label="ENTITY")])
ds = Dataset("test_ds", data=[e])
ds.apply_(operation)

=> Applying operation 'test' inplace
✔ Done
{'recon.v1.span_aliases': defaultdict(<class 'list'>, {2200217912058771699: ['new text']})}
KWARGS: 
{'spans_to_aliases_map': defaultdict(<class 'list'>, {2200217912058771699: ['new text']})}
✔ Completed operation 'test'


In [30]:
operation

In [31]:
ds.data

[Example(text='This is text', spans=[Span(text='text', start=8, end=12, label='ENTITY', token_start=None, token_end=None, kb_id='text')], tokens=None, meta={}, formatted=True, data=namespace(aliases=defaultdict(<class 'list'>, {2200217912058771699: ['new text']}))),
 Example(text='This is new text', spans=[Span(text='new text', start=8, end=16, label='ENTITY', token_start=None, token_end=None, kb_id='text')], tokens=None, meta={}, formatted=True, data=namespace(aliases=defaultdict(<class 'list'>, {2200217912058771699: ['new text']})))]

In [ ]:
@operation("recon.v1.augment.kb_expansion", factory=True)
def kb_expansion(
    example: Example,
    preprocessed_outputs: Dict[str, Any] = {},
    n_augs: int = 1,
    sub_prob: float = 0.5,
) -> List[Example]:

    spans_to_aliases_map = preprocessed_outputs["recon.v1.span_aliases"]
    print(preprocessed_outputs)
    print(example.data)

    def augmentation(span: Span, spans_to_aliases_map: Dict[int, List[str]]) -> Optional[str]:
        if hash(span) in spans_to_aliases_map:
            aliases = spans_to_aliases_map[hash(span)]

            if len(aliases) > 0:
                rand_alias = np.random.choice(aliases)
                index = aliases.index(rand_alias)
                del spans_to_aliases_map[hash(span)][index]
                return rand_alias

    return augment_example(
        example,
        augmentation,
        n_augs=n_augs,
        sub_prob=sub_prob,
        spans_to_aliases_map=spans_to_aliases_map,
    )